In [1]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import pandas as pd
import phate
import math
import random
import gc
import scprep
from datetime import datetime, time
from matplotlib.animation import ImageMagickWriter
import matplotlib.animation as animation
import zipfile
from urllib.request import urlopen
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import gaussian_kde
from scipy.stats import kde
from scipy.stats import binned_statistic
from scipy.stats import f_oneway
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['pdf.fonttype'] = 42
print(sns.__version__)
from anndata import AnnData
import scanpy as sc
from delve import *
import anndata as ad
from sklearn.preprocessing import MinMaxScaler
from kh import sketch
from sklearn.cluster import KMeans
import umap
print(sc.__version__)
today = datetime.now().strftime("%m%d%Y-%H%M")

0.11.2
1.9.1


In [2]:
#Read back in the subsampled adata file
adata_save_path = r'Z:\Garrett\4i_data\112723_RPE_31day_Etop\experiment\cell_data\standard_trimmed_noPSTAT5_adata_sub.h5ad'
standard_trimmed_noPSTAT5_adata_sub = anndata.read_h5ad(adata_save_path)

c:\Users\gases\.conda\envs\python_3_7\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
# Adjusted list comprehension to exclude var_names containing "total"
columns_to_keep = [name for name in standard_trimmed_noPSTAT5_adata_sub.var_names if "total_nuc_protein" not in name]

# Selecting the data with only the columns to keep
standard_trimmed_noPSTAT5_noTotal_adata_sub = standard_trimmed_noPSTAT5_adata_sub[:, columns_to_keep]


In [4]:
def laplacian_score_fs(adata = None,
                    k: int  = None,
                    n_jobs: int  = -1):

    X, feature_names, obs_names = parse_input(adata)
    W = construct_affinity(X = X, k = k, n_jobs = n_jobs)
    scores = laplacian_score(X = X, W = W)
    predicted_features = pd.DataFrame(scores, index = feature_names, columns = ['laplacian_score'])
    predicted_features = predicted_features.sort_values(by = 'laplacian_score', ascending = True)

    return predicted_features 

In [14]:
#l_score_fullest = laplacian_score_fs(adata_fullest_sub, k = 200)
l_score_standard = laplacian_score_fs(standard_trimmed_noPSTAT5_adata_sub, k = 100)
#l_score_normalized = laplacian_score_fs(normalized_trimmed_noPSTAT5_adata_sub, k = 200)

In [15]:
len(l_score_standard)

84

In [7]:
l_score_standard.index[:46]

Index(['R5_p38_nuc_mean', 'R0_pRb_nuc_mean', 'R6_pSTAT3_ring_mean',
       'R2_IL6_ring_mean', 'R0_pRb_ratio', 'R6_p65_ring_mean',
       'R6_pSTAT3_nuc_mean', 'R5_p38_ring_mean', 'R6_p65_nuc_mean',
       'R4_IL8_ring_mean', 'R0_Rb_ratio', 'R1_p21_nuc_mean', 'R2_IL6_nuc_mean',
       'R7_GATA4_nuc_mean', 'R1_p21_ratio', 'R0_Rb_nuc_mean',
       'R7_GATA4_ring_mean', 'nuc_area', 'R5_p38_ratio', 'R7_pp38_nuc_mean',
       'R2_pp65_ring_mean', 'R1_p16_nuc_mean', 'R3_p53_ring_mean',
       'R2_pp65_nuc_mean', 'R3_PLAUR_ring_mean', 'R3_PLAUR_nuc_mean',
       'R1_Jak2_nuc_mean', 'R7_pp38_ring_mean', 'R5_STAT5_ratio',
       'R5_STAT5_nuc_mean', 'R6_p65_ratio', 'R7_pp38_ratio',
       'R1_p21_ring_mean', 'R1_p16_ring_mean', 'R4_PARP1_nuc_mean',
       'R6_pSTAT3_ratio', 'minor_axis_length', 'R5_STAT5_ring_mean',
       'R3_p53_nuc_mean', 'R1_Jak2_ratio', 'R2_IL6_ratio', 'R1_Jak2_ring_mean',
       'R2_pp65_ratio', 'R4_PARP1_ratio', 'R7_GATA4_ratio',
       'major_axis_length'],
      dtype=

For plotting different data onto 3D PHATE structures

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
from pptx import Presentation
from pptx.util import Inches, Cm
from io import BytesIO

# Define angles for rotation in degrees
angles = [(30, 30), (30, 120), (30, 210), (30, 300)]

# Ensure the directory for saving plots exists
plots_dir = 'phate_plots'
os.makedirs(plots_dir, exist_ok=True)

# Create a PowerPoint presentation object
prs = Presentation()
slide_width = prs.slide_width
slide_height = prs.slide_height

# Get all features in .var_names
features = standard_trimmed_noPSTAT5_adata_sub.var_names

# Loop over each feature
for feature in features:
    # Obtain expression data for the feature
    data = standard_trimmed_noPSTAT5_adata_sub[:, feature].X

    # Cap high and low values for visualization
    percentile_99 = np.percentile(data, 99)
    percentile_1 = np.percentile(data, 1)
    data_capped = np.clip(data, percentile_1, percentile_99)

    # Normalize and map data to colors
    norm = plt.Normalize(np.min(data_capped), np.max(data_capped))
    cmap = plt.get_cmap('viridis')
    colors = cmap(norm(data_capped))

    # Create a single figure with a 2x2 grid of subplots
    fig = plt.figure(figsize=(16, 16))
    for i, (elev, azim) in enumerate(angles, start=1):
        ax = fig.add_subplot(2, 2, i, projection='3d')
        # Plot the feature data
        ax.scatter(standard_trimmed_noPSTAT5_adata_sub.obsm['X_phate'][:, 0], 
                   standard_trimmed_noPSTAT5_adata_sub.obsm['X_phate'][:, 1], 
                   standard_trimmed_noPSTAT5_adata_sub.obsm['X_phate'][:, 2], 
                   color=colors, label=feature, s=5)
        ax.view_init(elev=elev, azim=azim)
        ax.set_title(f'View Angle {i}: {feature}')

    # Save plot to a BytesIO object
    canvas = FigureCanvas(fig)
    canvas.draw()
    image_stream = BytesIO()
    plt.savefig(image_stream, format='png')
    plt.close(fig)

    # Add a slide to the presentation and remove default placeholders
    slide_layout = prs.slide_layouts[5]  # Using a blank slide layout
    slide = prs.slides.add_slide(slide_layout)
    for placeholder in slide.placeholders:
        placeholder.element.getparent().remove(placeholder.element)
    
    # Calculate image size to maximize slide use while keeping aspect ratio
    aspect_ratio = fig.get_figwidth() / fig.get_figheight()
    slide_aspect_ratio = slide_width / slide_height
    if aspect_ratio > slide_aspect_ratio:
        width = slide_width
        height = slide_width / aspect_ratio
    else:
        height = slide_height
        width = slide_height * aspect_ratio

    left = (slide_width - width) / 2
    top = (slide_height - height) / 2

    pic = slide.shapes.add_picture(image_stream, left, top, width=width, height=height)
    image_stream.close()

# Save the presentation
prs.save(f'{plots_dir}/phate_plots_presentation.pptx')
print("PowerPoint presentation saved with combined 3D PHATE plots optimized to fill slides.")
